Annotation creation

Removing - counties that have lesss tha 4 census
Sorting 0 labels based on H1 data points they have!

H1 data points analysis - add labels

In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
# import the csv
df = pd.read_csv('/home/h6x/git_projects/ornl-svi-data-processing/experiment_2/outputs/census_complex_info.csv')

In [3]:
df.head()

,State,STCNTY,Variable,Census_count,H0_count,H1_count,H0_inf_count,H1_inf_count,H1_withou_inf_count,H0_withou_inf_count,census_count
0,GA,13009,EP_POV,9,9,0,1,0,0.0,8.0,9.0
1,GA,13009,EP_UNEMP,9,9,1,1,0,1.0,8.0,9.0
2,GA,13009,EP_NOHSDP,9,9,1,1,0,1.0,8.0,9.0
3,GA,13009,EP_UNINSUR,9,9,1,1,0,1.0,8.0,9.0
4,GA,13009,EP_AGE65,9,9,1,1,0,1.0,8.0,9.0


In [4]:
variables = df['Variable'].unique().tolist()
variables

['EP_POV',
 'EP_UNEMP',
 'EP_NOHSDP',
 'EP_UNINSUR',
 'EP_AGE65',
 'EP_AGE17',
 'EP_DISABL',
 'EP_SNGPNT',
 'EP_LIMENG',
 'EP_MINRTY',
 'EP_MUNIT',
 'EP_MOBILE',
 'EP_CROWD',
 'EP_NOVEH',
 'EP_GROUPQ']

In [5]:
# make the STCNTY column STR
df['STCNTY'] = df['STCNTY'].apply(lambda x: str(x))

In [6]:
# For STCNTY column, we need to add a leading zero if the length of the string is 4
df['STCNTY'] = df['STCNTY'].apply(lambda x: '0' + x if len(x) == 4 else x)

In [7]:
unque_counties = df['STCNTY'].unique().tolist()

In [8]:
# create a new dataframe
new_df = pd.DataFrame()

In [9]:
for county in unque_counties:    
    # get the data for the county
    county_data = df[df['STCNTY'] == county]

    # get the average H0_withou_inf_count
    avg_H0_withou_inf_count = county_data['H1_withou_inf_count'].mean()

    new_row = pd.DataFrame([{
                'State': county_data['State'].values[0], 
                'STCNTY': county_data['STCNTY'].values[0], 
                'Census_count': county_data['census_count'].mean(),
                'H1_withou_inf_count_avg': avg_H0_withou_inf_count
            }])

    new_df = pd.concat([new_df, new_row], ignore_index=True)

In [10]:
new_df.head()

,State,STCNTY,Census_count,H1_withou_inf_count_avg
0,GA,13009,9.0,0.733333
1,GA,13013,18.0,1.266667
2,GA,13015,15.0,0.466667
3,GA,13017,5.0,0.000000
4,GA,13019,6.0,0.000000


Lets add mortaliy info

In [11]:
overdose_df = pd.read_excel('/home/h6x/git_projects/ornl-svi-data-processing/raw_data/HepVu_County_Opioid_Indicators_05DEC22.xlsx',dtype={'GEO ID': str})

In [12]:
overdose_df = overdose_df[['GEO ID','County Name','Narcotic Overdose Mortality Rate 2018']]

In [13]:
overdose_df = overdose_df.rename(columns={'Narcotic Overdose Mortality Rate 2018': 'NOD_Rate','GEO ID':'STCNTY'})

In [14]:
# Calculate the min and max of the column
min_value = overdose_df['NOD_Rate'].min()
max_value = overdose_df['NOD_Rate'].max()

In [15]:
min_value, max_value

(-9.0, 114.4)

In [16]:
# Assuming negative overdose rate means this data is missing
len(overdose_df[overdose_df['NOD_Rate'] < 0])

89

In [17]:
# lets not consider the missing data
overdose_df = overdose_df[overdose_df['NOD_Rate'] >= 0]

In [18]:
overdose_df.shape

(3132, 3)

In [19]:
overdose_df['OD_class_90'] = pd.qcut(overdose_df['NOD_Rate'], q=[0,0.90, 1], labels=['0', '1'])
overdose_df['OD_class_95'] = pd.qcut(overdose_df['NOD_Rate'], q=[0,0.95, 1], labels=['0', '1'])
overdose_df['OD_class_80'] = pd.qcut(overdose_df['NOD_Rate'], q=[0,0.80, 1], labels=['0', '1'])

In [20]:
overdose_df = overdose_df.reset_index(drop=True)

In [21]:
overdose_df['STCNTY'] = overdose_df['STCNTY'].apply(lambda x: '0' + x if len(x) == 4 else x)

In [22]:
# add NOD_Rate and OD_class to the df based on the STCNTY
merged_df = pd.merge(new_df, overdose_df[['STCNTY','NOD_Rate','OD_class_90','OD_class_95','OD_class_80']], on='STCNTY', how='left')

In [23]:
merged_df

,State,STCNTY,Census_count,H1_withou_inf_count_avg,NOD_Rate,OD_class_90,OD_class_95,OD_class_80
0,GA,13009,9.0,0.733333,8.0,0,0,0
1,GA,13013,18.0,1.266667,18.4,0,0,0
2,GA,13015,15.0,0.466667,19.9,0,0,0
3,GA,13017,5.0,0.000000,8.7,0,0,0
4,GA,13019,6.0,0.000000,11.1,0,0,0
...,...,...,...,...,...,...,...,...
3135,HI,15009,34.0,1.400000,15.3,0,0,0
3136,HI,15005,1.0,0.000000,19.4,0,0,0
3137,DE,10001,32.0,1.933333,32.9,1,0,1
3138,DE,10003,129.0,12.333333,43.6,1,1,1


drop if census count less than 4

In [24]:
# drop if xcensus count less than 4
merged_df_ = merged_df[merged_df['Census_count'] >= 4]

In [25]:
merged_df_.shape

(2266, 8)

In [26]:
# get the county STCNTY into a list
counties_grater_than_4_census = merged_df_['STCNTY'].tolist()

Now check the generated data

In [27]:
BASE_DIR = '/home/h6x/git_projects/ornl-svi-data-processing/experiment_2/processed_data_1'
DATA_DIR = f"{BASE_DIR}/npy_all_variables"
COMBINED_FEATURES_DIR = f"{BASE_DIR}/npy_combined"

In [28]:
def get_npy_filenames(directory):
    """
    Get all .npy file names in the specified directory without the .npy extension.
    
    Args:
    directory (str): The path to the directory containing .npy files.
    
    Returns:
    list: A list of file names without the .npy extension.
    """
    file_names = []
    for file in os.listdir(directory):
        if file.endswith('.npy'):
            file_names.append(os.path.splitext(file)[0])
    return file_names

In [29]:
fips_codes_generated_persistence_images = get_npy_filenames(COMBINED_FEATURES_DIR)

In [30]:
len(fips_codes_generated_persistence_images)

2746

In [31]:
merged_df_.shape

(2266, 8)

In [32]:
# FILTER merged_df_ based on the fips_codes_generated_persistence_images
merged_df_filtered = merged_df_[merged_df_['STCNTY'].isin(fips_codes_generated_persistence_images)]

In [33]:
merged_df_filtered.shape

(2227, 8)

In [34]:
merged_df_filtered.head()

,State,STCNTY,Census_count,H1_withou_inf_count_avg,NOD_Rate,OD_class_90,OD_class_95,OD_class_80
0,GA,13009,9.0,0.733333,8.0,0,0,0
1,GA,13013,18.0,1.266667,18.4,0,0,0
2,GA,13015,15.0,0.466667,19.9,0,0,0
3,GA,13017,5.0,0.000000,8.7,0,0,0
4,GA,13019,6.0,0.000000,11.1,0,0,0


In [35]:
high_df = merged_df_filtered[merged_df_filtered['OD_class_90'] == '1']

In [36]:
high_df.shape

(269, 8)

In [37]:
low_df = merged_df_filtered[merged_df_filtered['OD_class_90'] == '0']

In [39]:
# sort the low_df based on the H0_withou_inf_count_avg
low_df = low_df.sort_values(by='H1_withou_inf_count_avg', ascending=False)

In [40]:
low_df.head()

,State,STCNTY,Census_count,H1_withou_inf_count_avg,NOD_Rate,OD_class_90,OD_class_95,OD_class_80
2278,CA,06037,2315.0,271.666667,9.8,0,0,0
3035,IL,17031,1315.0,156.600000,23.4,0,0,0
987,AZ,04013,908.0,117.533333,24.5,0,0,1
313,TX,48201,783.0,90.066667,12.7,0,0,0
182,NY,36047,749.0,80.266667,11.7,0,0,0


In [42]:
# get 269 rows and print the tail
low_df = low_df.iloc[:269]

low_df.tail()

,State,STCNTY,Census_count,H1_withou_inf_count_avg,NOD_Rate,OD_class_90,OD_class_95,OD_class_80
610,PA,42013,34.0,3.200000,20.7,0,0,0
1923,MT,30111,32.0,3.200000,17.9,0,0,0
2942,LA,22019,42.0,3.200000,18.5,0,0,0
705,FL,12097,41.0,3.133333,21.5,0,0,0
203,NY,36091,50.0,3.133333,11.4,0,0,0


In [43]:
# combine the high_df and low_df
combined_df = pd.concat([high_df, low_df], ignore_index=True)

In [44]:
combined_df.shape

(538, 8)

In [45]:
# drop OD_class_95 and OD_class_80
combined_df = combined_df.drop(columns=['OD_class_95','OD_class_80'])

In [46]:
# output annotation path
OUTPUT_ANNOTATION_PATH = f'{BASE_DIR}/annotations_2018_npy_2_classes_only_h0h1_90_percentile.csv'

In [48]:
# Save the filtered annotation dataframe
# combined_df.to_csv(OUTPUT_ANNOTATION_PATH, index=False)
combined_df.to_csv(OUTPUT_ANNOTATION_PATH, index=False)